In [1]:
#加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
#定义超参数
BATCH_SIZE=128
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#cpu训练或者gpu训练
EPOCHS=10#训练次数

In [3]:
#构建pipeline,对图像处理
pipeline=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))])

In [4]:
#下载，加载数据集
from torch.utils.data import dataloader
train_set=datasets.MNIST("data",train=True,download=True,transform=pipeline)
test_set=datasets.MNIST("data",train=False,download=True,transform=pipeline)
#加载数据
train_loader=dataloader.DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
test_loader=dataloader.DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)

In [5]:
#插入代码，显示MNIST数据集的图片
with open("./data/MNIST/raw/train-images-idx3-ubyte","rb") as f:
    file=f.read()

In [6]:
# image1=[int(str(item).encode('ascii'),16) for item in file[16:800]]
# print(image1)

In [7]:
# import cv2
# import numpy as np

# image1_np=np.array(image1,dtype=np.uint8).reshape(28,28,1)
# print(image1_np.shape)

# cv2.imwrite("digit.jpg",image1_np)

In [8]:
#构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,10,5)#灰度图片通道， 输出通道，卷积核大小
        self.conv2=nn.Conv2d(10,20,3)
        self.fcl=nn.Linear(20*10*10,500)
        self.fc2=nn.Linear(500,10)#输入通道，输出通道

    def forward(self,x):
        input_size=x.size(0)
        x=self.conv1(x)
        x=F.relu(x)
        x=F.max_pool2d(x,2,2)

        x=self.conv2(x)
        x=F.relu(x)

        x=x.view(input_size,-1)#数据降维

        x=self.fcl(x)
        x=F.relu(x)

        x=self.fc2(x)

        output=F.log_softmax(x,dim=1)

        return output


In [9]:
#定义优化器
model=Digit().to(DEVICE)
optimizer=optim.Adam(model.parameters())

In [10]:
#定义训练模型
def train_model(model,device,train_loader,optimizer,epoch):
    #训练模型
    model.train()
    for batch_index,(data,target) in enumerate(train_loader):
        #将数据放到device上
        data,target=data.to(device),target.to(device)
        #梯度初始化为零
        optimizer.zero_grad()
        #训练后的结果
        output=model(data)
        #计算损失
        loss=F.nll_loss(output,target)
        #反向传播
        loss.backward()
        #更新参数
        optimizer.step()
        if batch_index%3000==0:
            print("Train Epoch:{}[{}/{}({:.0f}%)]\tLoss:{:.6f}".format(
                epoch,batch_index*len(data),len(train_loader.dataset),
                100.*batch_index/len(train_loader),loss.item()
            ))

In [11]:
#定义测试方法
def test_model(model,device,test_loader):
    #测试模型
    model.eval()
    #测试损失
    test_loss=0
    #正确率
    correct=0
    with torch.no_grad():#不进行梯度计算，也不进行反向传播
        for data,target in test_loader:
            #将数据放到device上
            data,target=data.to(device),target.to(device)
            output=model(data)
            #计算损失
            test_loss+=F.nll_loss(output,target,reduction="sum").item()
            pred=output.max(1,keepdim=True)[1]
            #计算正确率 
            correct+=pred.eq(target.view_as(pred)).sum().item()
    test_loss/=len(test_loader.dataset)
    print("\nTest set:Average loss:{:.4f},Accuracy:{}/{}({:.0f}%)\n".format(
        test_loss,correct,len(test_loader.dataset),
        100.*correct/len(test_loader.dataset)
    ))



In [12]:
#调用模型输出方法
for epoch in range(1,EPOCHS+1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

Train Epoch:1[0/60000(0%)]	Loss:2.307493

Test set:Average loss:0.0535,Accuracy:9831/10000(98%)

Train Epoch:2[0/60000(0%)]	Loss:0.049108

Test set:Average loss:0.0417,Accuracy:9860/10000(99%)

Train Epoch:3[0/60000(0%)]	Loss:0.051144

Test set:Average loss:0.0422,Accuracy:9865/10000(99%)

Train Epoch:4[0/60000(0%)]	Loss:0.013760

Test set:Average loss:0.0316,Accuracy:9894/10000(99%)

Train Epoch:5[0/60000(0%)]	Loss:0.009258

Test set:Average loss:0.0350,Accuracy:9884/10000(99%)

Train Epoch:6[0/60000(0%)]	Loss:0.012312

Test set:Average loss:0.0329,Accuracy:9901/10000(99%)

Train Epoch:7[0/60000(0%)]	Loss:0.039418

Test set:Average loss:0.0385,Accuracy:9882/10000(99%)

Train Epoch:8[0/60000(0%)]	Loss:0.002975

Test set:Average loss:0.0383,Accuracy:9893/10000(99%)

Train Epoch:9[0/60000(0%)]	Loss:0.019145

Test set:Average loss:0.0390,Accuracy:9894/10000(99%)

Train Epoch:10[0/60000(0%)]	Loss:0.001883

Test set:Average loss:0.0413,Accuracy:9894/10000(99%)

